# РОБОТА З БІБЛІОТЕКОЮ OPENCV

## Мета: Освоїти навички практичного використання бібліотеки OpenCV для обробки даних

#### Обчислення варіанту

In [1]:
N = ord('M') % 5 + 1
print(N)

3


#### Зчитування з файлу

In [2]:
import pandas as pd

df = pd.read_excel('./lab5.xlsx')

df

,N,file name,image size,glasses color,line width
0,1,emma-watson2.jpg,300x300,red,2
1,2,emma-watson.jpg,400x400,blue,3
2,3,draco.jpg,700x700,cyan,4
3,4,longbottom.jpg,600x600,magenta,5
4,5,ron_wesley.jpg,500x500,yellow,6


#### Відбір даних за варіантом

In [3]:
my_df = df[df['N'] == N]
my_df

,N,file name,image size,glasses color,line width
2,3,draco.jpg,700x700,cyan,4


# Опис функції `process_image`

Функція `process_image` призначена для обробки зображень на основі даних, отриманих із датафрейму. Вона включає кілька кроків: обрізання зображення до вказаних розмірів, центрованих на обличчі, додавання окулярів, перенісся та дужок, а також збереження отриманого результату у файл.

## Параметри функції

- **dataframe** (тип: `pandas.DataFrame`): 
  - Датафрейм, який містить інформацію про:
    - `file name`: назва файлу зображення
    - `glasses color`: колір рамки окулярів
    - `line width`: товщина лінії окулярів
    - `image size`: бажаний розмір зображення у форматі `ширина x висота`

## Кроки виконання функції

### 1. Завантаження даних із датафрейму

```python
file_name = dataframe['file name'].values[0]
glasses_color = dataframe['glasses color'].values[0]
line_width = int(dataframe['line width'].values[0])
image_size = dataframe['image size'].values[0]
```

- Витягує значення із стовпців датафрейму для подальшої обробки.

### 2. Визначення розміру зображення

```python
width, height = map(int, image_size.split('x'))
```

- Розбиває рядок на ширину та висоту для подальшого використання при обрізанні зображення.

### 3. Завантаження зображення

```python
image_path = f'./Images/{file_name}'
img = cv2.imread(image_path)
```

- Після завантаження перевіряє, чи вдалося відкрити файл. Якщо ні — виводить помилку та завершує виконання.

### 4. Завантаження каскадів для виявлення обличчя та очей

```python
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
```

- Використовує каскади Haar для визначення обличчя та очей на зображенні.

### 5. Виявлення обличчя та очей

```python
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
```

- Перетворює зображення у відтінки сірого для полегшення виявлення об'єктів.

### 6. Центрування та обрізання зображення

```python
(x, y, w, h) = faces[0]
face_center_x = x + w // 2
face_center_y = y + h // 2
x1 = max(0, face_center_x - width // 2)
y1 = max(0, face_center_y - height // 2)
x2 = min(img.shape[1], x1 + width)
y2 = min(img.shape[0], y1 + height)
cropped_img = img[y1:y2, x1:x2]
```

- Вираховує координати для обрізання зображення, щоб центр обличчя був у центрі нової рамки.

### 7. Перетворення в формат PIL та малювання окулярів

- Конвертує обрізане зображення у формат `PIL`, щоб мати можливість малювати за допомогою `ImageDraw`.

### 8. Визначення очей на обрізаному зображенні

```python
roi_gray = gray[y1:y2, x1:x2]
eyes = eye_cascade.detectMultiScale(roi_gray)
```

- Виявляє очі лише у верхній половині обрізаного зображення.

### 9. Сортування та обчислення позицій для окулярів

```python
eyes = sorted(eyes, key=lambda eye: eye[0])
eye_centers = [(ex + ew // 2, ey + eh // 2) for (ex, ey, ew, eh) in eyes]
radii = [int(min(ew, eh) / 2) * 1.2 for (ex, ey, ew, eh) in eyes]
```

- Обчислює центри та радіуси для кожного ока.

### 10. Малювання окулярів та переносиці

```python
draw.ellipse([...])
draw.line([...])
```

- Малює круглі лінзи, перенісся та дужки.

### 11. Збереження результату

```python
output_path = './dynamic_size_image_with_glasses.png'
img_pil.save(output_path)
print(f"Зображення збережено у {output_path}")
```

- Зберігає оброблене зображення у файл.

## Підсумок
Функція **`process_image`** є комплексним інструментом для автоматизації обробки зображень:
- Визначає положення обличчя та очей.
- Додає стильні окуляри з переніссям та дужками.
- Центрує обличчя та обрізає зображення до потрібного розміру.

---

### Приклад використання
```python
process_image(my_df)
```

- Функція викликається для обробки даних з датафрейму `my_df`.

In [4]:
import cv2
from PIL import Image, ImageDraw
import numpy as np

def process_image(dataframe):
    """
    Функція приймає дані з датафрейму, обрізає зображення до заданого розміру з відцентрованим обличчям,
    додає окуляри з переніссям та дужками і зберігає результат у файл.
    """
    # Отримання значень з датафрейму
    file_name = dataframe['file name'].values[0]
    glasses_color = dataframe['glasses color'].values[0]
    line_width = int(dataframe['line width'].values[0])
    image_size = dataframe['image size'].values[0]

    # Отримання ширини та висоти з колонки 'image size'
    width, height = map(int, image_size.split('x'))

    # Завантаження зображення
    image_path = f'./Images/{file_name}'
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Не вдалося завантажити зображення {file_name}")
        return

    # Завантаження каскадів для обличчя та очей
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Перетворення зображення в відтінки сірого для визначення обличчя
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    if len(faces) == 0:
        print("Обличчя не знайдено")
        return

    # Робота з першим знайденим обличчям
    (x, y, w, h) = faces[0]

    # Обчислення центра обличчя
    face_center_x = x + w // 2
    face_center_y = y + h // 2

    # Обрізання зображення до вказаного розміру (width x height) з центруванням обличчя
    x1 = max(0, face_center_x - width // 2)
    y1 = max(0, face_center_y - height // 2)
    x2 = min(img.shape[1], x1 + width)
    y2 = min(img.shape[0], y1 + height)

    # Коригування координат, якщо вони виходять за межі зображення
    if x2 - x1 < width:
        x1 = max(0, x2 - width)
    if y2 - y1 < height:
        y1 = max(0, y2 - height)

    cropped_img = img[y1:y2, x1:x2]

    # Перетворення до формату PIL для малювання
    img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)

    # Визначення очей на обрізаному зображенні
    roi_gray = gray[y1:y2, x1:x2]
    eyes = eye_cascade.detectMultiScale(roi_gray)

    # Перевірка, чи знайдено два ока
    if len(eyes) < 2:
        print("Знайдено менше двох очей")
        return

    # Сортуємо очі за координатою X (щоб визначити ліве та праве око)
    eyes = sorted(eyes, key=lambda eye: eye[0])

    # Масштабування координат для очей та радіуси
    eye_centers = []
    radii = []
    for (ex, ey, ew, eh) in eyes:
        eye_center = (ex + ew // 2, ey + eh // 2)
        radius = int(min(ew, eh) / 2) * 1.2
        eye_centers.append(eye_center)
        radii.append(radius)

    # Малювання лінз окулярів
    draw.ellipse([
        (eye_centers[0][0] - radii[0], eye_centers[0][1] - radii[0]),
        (eye_centers[0][0] + radii[0], eye_centers[0][1] + radii[0])
    ], outline=glasses_color, width=line_width)

    draw.ellipse([
        (eye_centers[1][0] - radii[1], eye_centers[1][1] - radii[1]),
        (eye_centers[1][0] + radii[1], eye_centers[1][1] + radii[1])
    ], outline=glasses_color, width=line_width)

    # Малювання переносиці
    draw.line([
        (eye_centers[0][0] + radii[0], eye_centers[0][1]),
        (eye_centers[1][0] - radii[1], eye_centers[1][1])
    ], fill=glasses_color, width=line_width)

    # Малювання дужок
    draw.line([
        (eye_centers[0][0] - radii[0], eye_centers[0][1]),
        (eye_centers[0][0] - radii[0] - 40, eye_centers[0][1] - 20)
    ], fill=glasses_color, width=line_width)

    draw.line([
        (eye_centers[1][0] + radii[1], eye_centers[1][1]),
        (eye_centers[1][0] + radii[1] + 40, eye_centers[1][1] - 20)
    ], fill=glasses_color, width=line_width)

    # Збереження результату у файл
    output_path = './dynamic_size_image_with_glasses.png'
    img_pil.save(output_path)
    print(f"Зображення збережено у {output_path}")

# Викликаємо функцію для відфільтрованого датафрейму
process_image(my_df)


Зображення збережено у ./dynamic_size_image_with_glasses.png
